# Modulos

In [1]:
import time
import requests
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# Funciones

In [2]:
def extract_results(driver):
    dc= {"Title":[], "Link":[], "Description":[]}
    results = driver.find_elements(By.XPATH, "//td[@class='bmtextview']")
    
    for result in results:
        link_element = result.find_element(By.TAG_NAME, "a")
        description_element = result.find_element(By.CLASS_NAME, "bbm_result_fieldvalue")

        title = link_element.text
        link = link_element.get_attribute("href")
        description = description_element.text

        dc["Title"].append(title)
        dc["Link"].append(link)
        dc["Description"].append(description)

    return pd.DataFrame(dc)        

In [3]:
def download_file(url, file_name):

    response = requests.get(url)
    if response.status_code == 200:
        
        with open(file_name, 'wb') as file:
            file.write(response.content)
            print(f"Archivo descargado: {file_name}")
    else:
        print(f"Error al descargar el archivo: {response.status_code}")

# Lectura de datos

## Resultados Pagina Web

In [4]:
url = "https://www.banxico.org.mx/AplBusquedasBM2/bgenwww_es.jsp"
search_term = "tpv"

In [5]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

search_bar = driver.find_element(By.NAME, "_userquery")
search_bar.send_keys(search_term)
search_bar.send_keys(Keys.RETURN)
time.sleep(5)

ls_data = []
ls_data.append(extract_results(driver))

while True:
    try:
        next_button = driver.find_element(By.XPATH, "//a[contains(text(), 'Siguiente')]")
        next_button.click()
        time.sleep(5)  # Esperar a que los resultados carguen
        ls_data.append(extract_results(driver))
    except:
        break  # Salir del bucle si no hay más páginas

driver.quit()

The chromedriver version (129.0.6668.91) detected in PATH at C:\SeleniumDrivers\chromedriver.exe might not be compatible with the detected chrome version (130.0.6723.70); currently, chromedriver 130.0.6723.69 is recommended for chrome 130.*, so it is advised to delete the driver in PATH and retry


In [6]:
all_searchs = pd.concat(ls_data, ignore_index=True)

### Procesamiento

In [7]:
all_searchs["Type"] = all_searchs["Title"].apply(lambda x: (x.split(",")[-1]).strip())
all_searchs["Title"] = all_searchs["Title"].apply(lambda x: " ".join(x.split(",")[:-1]))
all_searchs.Type.unique()

array(['SIE', 'pdf', 'html'], dtype=object)

In [8]:
all_searchs.sample(5)

,Title,Link,Description,Type
32,Otros,https://www.banxico.org.mx/servicios/tasas-de-...,Otros. Otros. Tasas de descuento por giro de n...,pdf
127,CCM-EconLab_ficha,https://www.banxico.org.mx/apps/econlab/fichas...,CCM-EconLab_ficha. Ficha de datos en español d...,pdf
28,Entretenimiento,https://www.banxico.org.mx/servicios/tasas-de-...,Entretenimiento. Entretenimiento. Tasas de des...,pdf
58,Colegios y universidades,https://www.banxico.org.mx/servicios/tasas-de-...,Colegios y universidades. Colegios y universid...,pdf
62,Telecomunicaciones,https://www.banxico.org.mx/servicios/tasas-de-...,Telecomunicaciones. Telecomunicaciones. Tasas ...,pdf


In [56]:
all_searchs[all_searchs.Title=='Hoteles'].iloc[0,1]

'https://www.banxico.org.mx/servicios/tasas-de-descuento-para-tarjetas-de-credito-por-gi/%7B4B7FCB63-8D2C-F86D-578D-130144A51CD5%7D.pdf'

In [9]:
all_searchs["Download"] = all_searchs["Link"].apply(lambda x: 1 if "tasas-de-descuento" in x else 0)
all_searchs["Card"] = np.where(all_searchs["Download"]==1,
                               np.where(all_searchs["Link"].str.contains("credito"),"Credito","Debito"),"No Aplica")

In [10]:
all_searchs["File_name"] = np.where(all_searchs["Download"]==1,"Descuento " +all_searchs["Card"] +" " +all_searchs["Title"] +"."+ all_searchs["Type"], '' )

In [11]:
all_searchs[all_searchs.Download==1]

,Title,Link,Description,Type,Download,Card,File_name
18,Agregadores,https://www.banxico.org.mx/servicios/tasas-de-...,Agregadores. Agregadores. Tasas de descuento p...,pdf,1,Credito,Descuento Credito Agregadores.pdf
19,Agregadores,https://www.banxico.org.mx/servicios/tasas-de-...,Agregadores. Agregadores. Tasas de descuento p...,pdf,1,Debito,Descuento Debito Agregadores.pdf
21,Misceláneas,https://www.banxico.org.mx/servicios/tasas-de-...,Misceláneas. Misceláneas. Tasas de descuento p...,pdf,1,Credito,Descuento Credito Misceláneas.pdf
22,Restaurantes,https://www.banxico.org.mx/servicios/tasas-de-...,Restaurantes. Restaurantes. Tasas de descuento...,pdf,1,Credito,Descuento Credito Restaurantes.pdf
23,Ventas al detalle (Retail),https://www.banxico.org.mx/servicios/tasas-de-...,Ventas al detalle (Retail). Ventas al detalle ...,pdf,1,Credito,Descuento Credito Ventas al detalle (Retail).pdf
24,Misceláneas,https://www.banxico.org.mx/servicios/tasas-de-...,Misceláneas. Misceláneas. Tasas de descuento p...,pdf,1,Debito,Descuento Debito Misceláneas.pdf
25,Restaurantes,https://www.banxico.org.mx/servicios/tasas-de-...,Restaurantes. Restaurantes. Tasas de descuento...,pdf,1,Debito,Descuento Debito Restaurantes.pdf
26,Ventas al detalle (Retail),https://www.banxico.org.mx/servicios/tasas-de-...,Ventas al detalle (Retail). Ventas al detalle ...,pdf,1,Debito,Descuento Debito Ventas al detalle (Retail).pdf
27,Comida rápida,https://www.banxico.org.mx/servicios/tasas-de-...,Comida rápida. Comida rápida. Tasas de descuen...,pdf,1,Credito,Descuento Credito Comida rápida.pdf
28,Entretenimiento,https://www.banxico.org.mx/servicios/tasas-de-...,Entretenimiento. Entretenimiento. Tasas de des...,pdf,1,Credito,Descuento Credito Entretenimiento.pdf


### Salida

In [15]:
all_searchs.sort_values(by=["Download", "Title"], ascending=[False,True]).to_csv("all_searchs_banxico.csv", index=False)

## Descarga PDF's

In [12]:
folder = "PDFs/"
for index,row in all_searchs[all_searchs.Download==1].iterrows():
    
    url = row["Link"]
    file_name = folder + row["File_name"]
    download_file(url, file_name)

Archivo descargado: PDFs/Descuento Credito Agregadores.pdf
Archivo descargado: PDFs/Descuento Debito Agregadores.pdf
Archivo descargado: PDFs/Descuento Credito Misceláneas.pdf
Archivo descargado: PDFs/Descuento Credito Restaurantes.pdf
Archivo descargado: PDFs/Descuento Credito Ventas al detalle (Retail).pdf
Archivo descargado: PDFs/Descuento Debito Misceláneas.pdf
Archivo descargado: PDFs/Descuento Debito Restaurantes.pdf
Archivo descargado: PDFs/Descuento Debito Ventas al detalle (Retail).pdf
Archivo descargado: PDFs/Descuento Credito Comida rápida.pdf
Archivo descargado: PDFs/Descuento Credito Entretenimiento.pdf
Archivo descargado: PDFs/Descuento Credito Otros.pdf
Archivo descargado: PDFs/Descuento Debito Comida rápida.pdf
Archivo descargado: PDFs/Descuento Debito Entretenimiento.pdf
Archivo descargado: PDFs/Descuento Debito Otros.pdf
Archivo descargado: PDFs/Descuento Credito Médicos y dentistas.pdf
Archivo descargado: PDFs/Descuento Credito Refacciones y ferreterías.pdf
Archivo d